# SIZING REFRIGERANT LINES
---

This example is based on the example in ***TRANE, Air Conditioning Clinic: Refrigerant Piping (June 2011, TRG-TRC006-EN)***.

In [1]:
import warnings
from pathlib import Path
from hvac import Quantity
from hvac.fluids import Fluid, CoolPropWarning
from hvac.refrigerant_piping import (
    CopperTubing,
    RefrigerantCycleInfo,
    SuctionLineSizer,
    DischargeLineSizer,
    LiquidLineSizer
)

In [2]:
warnings.filterwarnings('ignore', category=CoolPropWarning)

Q_ = Quantity
R22 = Fluid('R22')

**Get copper tubes**

With script `copper_tubing.py`, in the same folder as this notebook, shelves with copper tube sizes were created. To get access to a shelf, set the file path to the shelf. To retrieve then the records on the shelf, call method `get_records()` of the `CopperTubing` class.

In [3]:
CopperTubing.db_path = Path("./copper-tube-ACR-D")
copper_tubes = CopperTubing.get_records()

**Set refrigeration cycle info**

To size the refrigeration lines, we need:
- the type of refrigerant
- the evaporation temperature
- the condensing temperature
- the amount of refrigerant superheating at the evaporator outlet
- the amount of refrigerant subcooling at the condenser outlet
- the temperature difference between the discharge gas at the compressor outlet and the condensing temperature, i.e. the amount of refrigerant superheating at the compressor outlet

In [4]:
cycle_info = RefrigerantCycleInfo(
    refrigerant=R22,
    T_evp=Q_(40, 'degF'),
    T_cnd=Q_(125, 'degF'),
    evp_superheat=Q_(12, 'delta_degF'),
    cnd_subcooling=Q_(15, 'delta_degF'),
    cmp_superheat=Q_(70, 'delta_degF')
)

**Set the maximum and minimum cooling capacity of the system**

In a refrigeration system where the cooling capacity can be adapted through compressor unloading or by turning multiple compressors on/off, the mass flow rate of refrigerant, and therefore, the flow velocity of refrigerant can vary between a minimum and maximum value. The minimum flow velocity should remain high enough to ensure proper oil return to the compressor(s). To check the minimum flow velocity, the minimum cooling capacity needs to be specified. (In case of a single fixed speed compressor or a compressor without capacity control -which are on/off controlled- we don't need to specify the minimum cooling capacity, as there can be only one refrigerant mass flow rate circulating in the machine when the compressor is running.)

In [5]:
Q_dot_evp_max = Q_(20, 'refrigeration_ton')
Q_dot_evp_min = Q_(10, 'refrigeration_ton')

Sizing the refrigerant lines is a two step process. 

First a "sizer" is instantiated. The "sizer" must be given a collection of copper tubes it can select from. Based on the refrigeration cycle info and the maximum and the minimum cooling capacity of the refrigeration system, the "sizer" determines which copper tubes are suitable by calculating the refrigerant flow velocity in each copper tube. The flow velocity must be high enough to ensure proper oil return, but if it is too high, objectionable noise and/or pipe wall erosion can arise. Should multiple copper tubes satisfy the minimum and maximum velocity criteria, it finally selects the copper tube with the smallest pressure drop.

Once the nominal diameter of the copper tube has been determined, fittings and accessories can be added to the line to calculate the final, total pressure loss along the line. When fittings are added to the line, their equivalent length of copper tube (which depends on the nominal diameter of the copper tube) needs to be specified, along with the number of times they are used in the line. It is also possible to specify any additional pressure losses across fittings/accessories directly.

In the case of a liquid line, the minimum flow velocity is not a concern as oil readily mixes with liquid refrigerant. Therefore, the minimum flow velocity (i.e. the flow velocity at the minimum cooling capacity) is not calculated. The copper tube with the smallest inside diameter that satisfies the maximum velocity criterion is returned. When the pressure drop along the liquid line is calculated, it is also checked if the refrigerant is still subcooled enough at the entrance of the expansion device. If this should be the case, a warning will be set on the output returned by method `pressure_drop(...)`.

## Suction Line

In [6]:
suction_line_sizer = SuctionLineSizer(
    copper_tubes=copper_tubes,
    cycle_info=cycle_info,
    Q_dot_evp_max=Q_dot_evp_max,
    Q_dot_evp_min=Q_dot_evp_min,
)

The suction line is divided in two sections: the horizontal section and the vertical riser are sized separately, as the vertical riser needs a higher minimum velocity to ensure proper oil return due to the opposing effect of gravity.

**Horizontal Section**

In [7]:
horizontal_section = suction_line_sizer.size(L=Q_(55, 'feet'))
if horizontal_section is None:
    print('no suitable copper tube available')
else:
    print(horizontal_section)

2 1/8 | maximum flow velocity = 9.5 m/s | minimum flow velocity = 4.8 m/s | maximum pressure drop = 4.502 kPa (0.250 K)


Knowing the nominal diameter of the copper tube, the fittings can be added with their appropriate equivalent length of copper tube: 

In [8]:
horizontal_section.pressure_drop(
    fittings=[
        ('long-radius-elbow', Q_(1.4, 'feet'), 3),
        ('short-radius-elbow', Q_(2.3, 'feet'), 3)
    ]
)
print(horizontal_section)

2 1/8 | maximum flow velocity = 9.5 m/s | minimum flow velocity = 4.8 m/s | maximum pressure drop = 5.399 kPa (0.300 K)


**Vertical Riser**

In [9]:
vertical_riser = suction_line_sizer.size(
    L=Q_(15, 'feet'),
    elevation=Q_(15, 'feet')
)
if vertical_riser is None:
    print('no suitable copper tube available')
else:
    print(vertical_riser)

1 5/8 | maximum flow velocity = 16.5 m/s | minimum flow velocity = 8.3 m/s | maximum pressure drop = 5.936 kPa (0.330 K)


## Discharge Line

In [10]:
discharge_line_sizer = DischargeLineSizer(
    copper_tubes=copper_tubes,
    cycle_info=cycle_info,
    Q_dot_evp_max=Q_dot_evp_max,
    Q_dot_evp_min=Q_dot_evp_min
)

In [11]:
discharge_line = discharge_line_sizer.size(L=Q_(50, 'feet'))
if discharge_line is None:
    print('no suitable copper tube available')
else:
    print(discharge_line)

1 3/8 | maximum flow velocity = 7.9 m/s | minimum flow velocity = 4.0 m/s | maximum pressure drop = 13.179 kPa (0.287 K)


In [12]:
discharge_line.pressure_drop(
    fittings=[('long-radius-elbow', Q_(0.9, 'feet'), 2)]
)
print(discharge_line)

1 3/8 | maximum flow velocity = 7.9 m/s | minimum flow velocity = 4.0 m/s | maximum pressure drop = 13.649 kPa (0.297 K)


## Liquid Line

In [13]:
liquid_line_sizer = LiquidLineSizer(
    copper_tubes=copper_tubes,
    cycle_info=cycle_info,
    Q_dot_evp_max=Q_dot_evp_max
)

In [14]:
liquid_line = liquid_line_sizer.size(
    L=Q_(100, 'feet'),
    elevation=Q_(4, 'feet')
)
if liquid_line is None:
    print('no suitable copper tube available')
else:
    print(liquid_line)

5/8 | maximum flow velocity = 2.7 m/s | minimum flow velocity = nan m/s | maximum pressure drop = nan kPa (nan K)


Add fittings and additional accessories to the liquid line and get the total pressure loss along the liquid line:

In [16]:
liquid_line.pressure_drop(
    fittings=[
        ('long-radius-elbow', Q_(0.4, 'feet'), 4),
        ('ball-valve', Q_(1.0, 'feet'), 2),
        ('sight-glass', Q_(1.0, 'feet'), 1)
    ],
    dP_add=[
        Q_(6, 'psi'),  # filter drier
        Q_(4, 'psi')   # solenoid valve
    ]
)
print(liquid_line)

5/8 | maximum flow velocity = 2.7 m/s | minimum flow velocity = nan m/s | maximum pressure drop = 218.188 kPa (4.965 K)


If no warning is displayed when printing the liquid line, it means that enough subcooling remains at the entrance of the expansion device.